In [81]:
import pandas as pd
import numpy as np

In [82]:
#Read in the datafile (data grabbed from API using 'get_data.py')
df_cityData = pd.read_csv('2017_NYC_Data.csv', low_memory=False)
print("Finished! Data now available.")

Finished! Data now available.


Clean up "Unspecified" borough's using dataset below (contains zipcode:borough pair):
https://www.nycbynatives.com/nyc_info/new_york_city_zip_codes.php

I copy and pasted this data into an excel spreadsheet and saved it as an csv file. There are other ways to grab the data although I found this way to be the easiest and quickest.

In [83]:
#filter the "Unspecified" boroughs and their coordinating incident_zip from 'df_cityData'
df_cleanedCityData = df_cityData[['borough', 'incident_zip']].dropna()
df_cleanedCityData = df_cleanedCityData.loc[df_cleanedCityData['borough']=='Unspecified']

In [84]:
df_nycZipBorough = pd.read_csv('NYC_Zips.csv') #read in the dataset specified in the markdown above

#clean up the new data and format it correctly
df_nycZipBorough = df_nycZipBorough.dropna()
df_nycZipBorough['zipcode'] = df_nycZipBorough['zipcode'].apply(int).apply(str)#remove the '.0' from zips(.replace method was acting weird)
df_nycZipBorough['borough'] = df_nycZipBorough['borough'].str.upper().replace('STATEN', 'STATEN ISLAND')
df_nycZipBorough.rename(columns={'zipcode': 'incident_zip'} ,inplace=True)

#merge the two dataframes (containing the unspecified boroughs and the specified ones on the 'incident_zip'),
#dropping the "Unspecified" borough row in the process
nowSpecifiedBoroughs = pd.merge(df_cleanedCityData['incident_zip'].to_frame(), df_nycZipBorough, on='incident_zip').drop_duplicates()

In [85]:
#merge the original cityData with the data containing the specified boroughs
df_cityData = pd.merge(df_cityData, nowSpecifiedBoroughs, how='left', on='incident_zip')

In [86]:
#Swap the "Unspecified" boroughs out with the 
df_cityData['borough_x']=df_cityData['borough_x'].replace('Unspecified', np.NaN)
df_cityData.borough_x.fillna(df_cityData.borough_y, inplace=True) #fill the now 'NaN' field with the specified borough value (borough_y)
df_cityData = df_cityData.drop(columns=['borough_y']) #drop the no longer needed row
df_cityData.rename(columns={'borough_x': 'borough'} ,inplace=True)

##Consider only the 10 most common overall complaint types. For each borough, how many of each of those 10 types were there in 2017?

filter out the top 10 complaint types

In [87]:
df_topTenComplaintTypes = df_cityData['complaint_type'].value_counts().head(10)
df_topTenComplaintTypes

Noise - Residential        230294
HEAT/HOT WATER             213530
Illegal Parking            146207
Blocked Driveway           136168
Street Condition            93314
Street Light Condition      84120
UNSANITARY CONDITION        79263
Noise - Street/Sidewalk     73139
Water System                65033
Noise                       60180
Name: complaint_type, dtype: int64

filter the city data further using the above results

In [88]:
#filter the records for which the complaint type matches one found in 'df_topTenComplaintTypes'
df_TopTenData = df_cityData.loc[df_cityData['complaint_type'].isin(df_topTenComplaintTypes.index)]
#filter the records to include only the 'borough' & 'complaint_type' (multiple for each borough), and the number of occurences for each complaint within that borough
df_TopTenData[['borough','complaint_type']].groupby(['borough','complaint_type'])['complaint_type'].count()

borough        complaint_type         
BRONX          Blocked Driveway           24601
               HEAT/HOT WATER             68718
               Illegal Parking            16147
               Noise                       3133
               Noise - Residential        57720
               Noise - Street/Sidewalk    14049
               Street Condition           11762
               Street Light Condition     18416
               UNSANITARY CONDITION       24535
               Water System               10179
BROOKLYN       Blocked Driveway           49421
               HEAT/HOT WATER             66987
               Illegal Parking            55505
               Noise                      15424
               Noise - Residential        67702
               Noise - Street/Sidewalk    21328
               Street Condition           25450
               Street Light Condition     22435
               UNSANITARY CONDITION       26667
               Water System               19805
M

##Consider only the 10 most common overall complaint types.  For the 10 most populous zip codes, how many of each of those 10 types were there in 2017?

read in the new dataset and merge the population data with the city data

In [89]:
#read in the dataset containing the population data
df_populationData = pd.read_csv('PopulationData.csv')

In [90]:
#merge the two dataframes
df_populationData.columns = ['incident_zip','population'] #rename columns to match other df
df_populationData['incident_zip'] = df_populationData['incident_zip'].apply(str)#change column type to string to handle 'NaN' in df_zipAndComplaintType
df_zipAndComplaintType = df_cityData[['incident_zip','complaint_type']] #grab necessary data from df_cityData
df_mergedData = pd.merge(df_populationData, df_zipAndComplaintType, on='incident_zip')

apply filtering and grab the necessary data

In [91]:
df_overlappingZips = df_mergedData.loc[df_mergedData['incident_zip'].isin(df_cityData['incident_zip'])] #filter for overlapping zip codes
df_overlappingZips = df_overlappingZips.loc[df_overlappingZips['complaint_type'].isin(df_topTenComplaintTypes.index)] #filter for top ten complaints
df_overlappingZips = df_overlappingZips.groupby(['incident_zip','population','complaint_type'])['complaint_type'].count()
df_overlappingZips = df_overlappingZips.to_frame('').sort_values(by='population', ascending=False).head(100) #sort and grab first 100 (10zips x 10complaints = 100rows)
print(df_overlappingZips.to_string()) #print ALL data rows 

                                                     
incident_zip population complaint_type               
11368        109931     UNSANITARY CONDITION      639
                        Street Light Condition    441
                        Blocked Driveway         4387
                        HEAT/HOT WATER           1621
                        Illegal Parking          1251
                        Noise                     158
                        Noise - Residential      2461
                        Street Condition          561
                        Noise - Street/Sidewalk   685
                        Water System              618
11226        101572     Noise - Residential      4857
                        Blocked Driveway         2204
                        Water System              406
                        Illegal Parking          1078
                        Noise                     440
                        HEAT/HOT WATER           7569
                        Nois

##Considering all complaint types. Which boroughs are the biggest "complainers" relative to the size of the population in 2017? Meaning, calculate a complaint-index that adjusts for population of the borough.

grab the total # of complaints for each borough

In [92]:
df_complaintCountPerBorough = df_cityData[['borough','complaint_type']].groupby(['borough'])['complaint_type'].count().reset_index()
df_complaintCountPerBorough.rename(columns = {'complaint_type': 'complaint_count'}, inplace=True)
df_complaintCountPerBorough

,borough,complaint_count
0,BRONX,450986
1,BROOKLYN,771796
2,MANHATTAN,480642
3,QUEENS,590107
4,STATEN ISLAND,129530


grab the total population of each borough

In [93]:
#Grab necessary data from df_cityData
df_zipBorough = df_cityData[['borough','incident_zip']]
df_zipBorough = df_zipBorough.replace(['0', 'N/A', 'UNKNOWN', 'NA','.',''], pd.np.nan).dropna().drop_duplicates() #filter out 'bad' rows

#Grab necessary data from df_populationData
df_zipPop = df_populationData[['incident_zip','population']]

#Merge and manipulate data to get the total population value for each borough
df_populationOfBoroughs = pd.merge(df_zipBorough, df_zipPop, on='incident_zip')
df_populationOfBoroughs = df_populationOfBoroughs.sort_values(by='borough')
df_populationOfBoroughs = df_populationOfBoroughs.groupby(['borough'])['population'].sum().reset_index()
df_populationOfBoroughs

,borough,population
0,BRONX,1592084
1,BROOKLYN,2732303
2,MANHATTAN,2073659
3,QUEENS,2755424
4,STATEN ISLAND,468730


merge the two tables shown above

In [94]:
df_biggestComplainers = pd.merge(df_complaintCountPerBorough, df_populationOfBoroughs, on='borough')
#calculate complaint index (complaint_count/population) for each borough
df_biggestComplainers['complaint_index'] = df_biggestComplainers['complaint_count']/df_biggestComplainers['population']
df_biggestComplainers.sort_values(by='complaint_index', ascending=False) #sort by biggest "complainers"

,borough,complaint_count,population,complaint_index
0,BRONX,450986,1592084,0.283268
1,BROOKLYN,771796,2732303,0.282471
4,STATEN ISLAND,129530,468730,0.276342
2,MANHATTAN,480642,2073659,0.231784
3,QUEENS,590107,2755424,0.214162


RESULT: Based on the above data, we can easily see that people from the Bronx are the biggest "complainers" with an average of around 1 complaint per 3.5 people. 